In [ ]:
from query.datasets.prelude import *
from timeit import default_timer as now

q1 = FaceGender.objects.values('id')
q2 = FaceGender.objects.values('id', 'face__person__frame__video__show')    

In [ ]:
def profile(s, f):
    start = now()
    f()
    print('{}: {:.3f}'.format(s, now() - start))
    
profile('nojoin', lambda: list(q1))
profile('withjoin', lambda: list(q2))

In [ ]:
q1.print_sql()
print('')
q1.explain()

In [ ]:
q2.print_sql()
print('')
q2.explain()